<a href="https://colab.research.google.com/github/akriegman/Wormhole-Geodesic/blob/master/worm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

def bandGEPP(A, b, depth):
  #solves the system Ax = b when A has no elements depth below or above the diagonal
  #so for a diagonal matrix depth = 1
  #does not check for good input values, sorry
  #runs in O(n*depth^2)
  #postcondition: A and b are modified
  
  n = len(A)
  
  #row reduce
  for col in range(n-1):
    pivot = np.argmax([abs(r[col]) for r in A[col:col+depth]]) + col
    
    if not pivot == col:
      A[col], A[pivot] = A[pivot], A[col]
      b[col], b[pivot] = b[pivot], b[col]
    
    for row in range(col+1, min(col+depth, n)):
      ratio = A[row][col] / A[col][col]
      for i in range(col, min(col+2*depth-1, n)):
        A[row][i] -= A[col][i] * ratio
      b[row] -= b[col] * ratio
  
  #back substitute
  x = []
  for i in range(-1, -n-1, -1):
    y = b[i]
    for j in range(1, min(depth+1, -i)):
      y -= A[i][i+j] * x[j-1]
    y /= A[i][i]
    x.insert(0, y)
  
  return x
      

In [0]:
def NRworm(l, a, b):
  #given a path, uses Newton-Raphson method to aproximate the geodesic
  #specialized for a wormhole with metric ds^2 = dl^2 + (b^2 + l^2)da^2
  #keeps first and last point constant
  #for more information see https://cs.stanford.edu/people/jbaek/18.821.paper2.pdf
  
  n = len(l)
  if not len(a) == n:
    return 0, 0
  
  #create the vector H(X) - X with alternating l's and a's
  X = [item for pair in zip(l, a) for item in pair][2:-2]
  
  Hl = [(l[k-1] + l[k+1])/2 - l[k] * (a[k+1] - a[k-1])**2/4
       for k in range(1, n-1)]
  Ha = [(a[k-1] + a[k+1])/2 + l[k] / (b**2 + l[k]**2) * (l[k+1] - l[k-1]) * (a[k+1] - a[k-1])/2
       for k in range(1, n-1)]
  HX = [item for pair in zip(Hl, Ha) for item in pair]
  
  B = [HX[i] - X[i] for i in range(2*n-4)]
  
  #setup the matrix dH(X) - I
  A = [[0 for j in range(2*n-4)] for i in range(2*n-4)]
  
  for i, k in zip(range(0, 2*n-6, 2), range(1, n-2)):
    A[i+2][i], A[i][i+2] = 0.5, 0.5
    A[i+2][i+1] = l[k+1] * a[k] * (a[k+2]-a[k])/2
    A[i][i+3] = -l[k] * a[k+1] * (a[k+1]-a[k-1])/2
    A[i+1][i+3] = 0.5 + l[k] / (b*b + l[k]*l[k]) * (l[k+1] - l[k-1])/2
    A[i+3][i+1] = 0.5 - l[k+1] / (b*b + l[k+1]*l[k+1]) * (l[k+2] - l[k])/2
    A[i+1][i+2] = l[k] / (b*b + l[k]*l[k]) * (a[k+1] - a[k-1])/2
    A[i+3][i] = -l[k+1] / (b*b + l[k+1]*l[k+1]) * (a[k+2] - a[k])/2
  
  for i, k in zip(range(0, 2*n-4, 2), range(1, n-1)):
    A[i][i] = -1 - (a[k+1] - a[k-1])**2/4
    A[i+1][i] = (b*b - l[k]*l[k]) / (b*b + l[k]*l[k])**2 * (l[k+1] - l[k-1]) * (a[k+1] - a[k-1])/2
    A[i+1][i+1] = -1
    
  Z = bandGEPP(A, B, 4)
  Z = [0,0] + Z + [0,0]
  L = [l[k] - Z[2*k] for k in range(n)]
  A = [a[k] - Z[2*k+1] for k in range(n)]
  return L, A

In [0]:
def iterworm(l, a, b):
  #different technique using functional iteration.
  #for more information see https://cs.stanford.edu/people/jbaek/18.821.paper2.pdf
  n = len(l)
  Hl = [(l[k-1] + l[k+1])/2 - l[k] * (a[k+1] - a[k-1])**2/4
       for k in range(1, n-1)]
  Ha = [(a[k-1] + a[k+1])/2 + l[k] / (b**2 + l[k]**2) * (l[k+1] - l[k-1]) * (a[k+1] - a[k-1])/2
       for k in range(1, n-1)]
  Hl.insert(0, l[0])
  Hl.append(l[-1])
  Ha.insert(0, a[0])
  Ha.append(a[-1])
  return Hl, Ha

In [52]:
m = [[0.0000002, 0.0000001, 0, 0],
    [1, 2, 1, 0],
    [0, 1, 2, 1],
    [0, 0, 10000000, 20000000]]

v = [0.0000002, 1, 4, 80000000]

bandGEPP(m, v, 2)


[1.0, 0.0, 0.0, 4.0]

In [117]:
import matplotlib.pyplot as plt
import math
b=1
p = [8,0]
q = [8, math.pi]
N = 20
l = [(k*p[0] + (N-k)*q[0])/N for k in range(N+1)]
a = [(k*p[1] + (N-k)*q[1])/N for k in range(N+1)]

for i in range(100):
  l, a = iterworm(l, a, b)

for i in range(0):
  l, a = NRworm(l, a, b)

x = [math.cos(a[k])*math.sqrt(b**2 + l[k]**2) for k in range(N+1)]
y = [math.sin(a[k])*math.sqrt(b**2 + l[k]**2) for k in range(N+1)]
plt.scatter(x, y)
plt.show

OverflowError: ignored

In [95]:
# m = [1,2]
m.insert(0,-1)
m.append(2)
m

[-1, 1, 2, 2]